## Writing ome.zarr data from a CZI image file

* Read the CZI image and its metadata into an 6D array
* reduce dimensionality to a 5D array
* write array into an OME-ZARR file

In [1]:
# check if the notebook runs in Google Colab
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [2]:
if IN_COLAB:
  # Install dependencies
  ! pip install --upgrade pip
  ! pip install czitools
  ! pip install ome-zarr
  ! pip install ngff-zarr[validate, dask-image]

In [3]:
from czitools.read_tools import read_tools
from czitools.metadata_tools import czi_metadata as czimd
import ngff_zarr as nz
from pathlib import Path
import dask.array as da
import zarr
import os
import requests
import ome_zarr.writer
import ome_zarr.format
from ome_zarr.io import parse_url
from typing import Union
import shutil
import numpy as np
from czitools.utils import logging_tools
from importlib.metadata import version

logger = logging_tools.set_logging()

# show currently used version of NGFF specification
ngff_version = ome_zarr.format.CurrentFormat().version
logger.info(f"Using ngff format version: {ngff_version}")
logger.info(f"ZARR Version: {zarr.__version__}")
logger.info(f"NGFF-ZARR Version: {nz.__version__}")
logger.info(f"OME-ZARR Version: {version('ome-zarr')}")

2025-08-25 15:17:31,259 - czitools - INFO - Using ngff format version: 0.5
2025-08-25 15:17:31,260 - czitools - INFO - ZARR Version: 3.1.1
2025-08-25 15:17:31,260 - czitools - INFO - NGFF-ZARR Version: 0.16.1
2025-08-25 15:17:31,262 - czitools - INFO - OME-ZARR Version: 0.12.2


In [4]:
def write_omezarr(
    array5d: Union[np.ndarray, da.Array],
    zarr_path: str,
    axes: str = "tczyx",
    overwrite: bool = False,
) -> str:
    """
     Writes a 5D array to an OME-ZARR file.
    Parameters:
    -----------
    array5d : Union[np.ndarray, da.Array]
        The 5D array to be written. The dimensions should not exceed 5.
    zarr_path : str
        The path where the OME-ZARR file will be saved.
    axes : str, optional
        The order of axes in the array. Default is "tczyx".
    overwrite : bool, optional
        If True, the existing file at zarr_path will be overwritten. Default is False.
    Returns:
    --------
    str
        The path to the written OME-ZARR folder if successful, otherwise None.
    Notes:
    ------
    - The function ensures the axes are in lowercase and removes any invalid dimensions.
    - If the zarr_path already exists and overwrite is True, the existing directory will be removed.
    - The function logs the NGFF format version being used.
    - The function writes the image data to the specified zarr_path.
    - If the writing process is successful, the function returns the zarr_path; otherwise, it returns None.
    """

    # check number of dimension of input array
    if len(array5d.shape) > 5:
        logger.warning("Input array as more than 5 dimensions.")
        return None

    # make sure lower case is use for axes order
    axes = axes.lower()

    # check for invalid dimensions and clean up
    for character in ["b", "h", "s", "i", "v", "a"]:
        axes = axes.replace(character, "")

    # check if zarr_path already exits
    if Path(zarr_path).exists() and overwrite:
        shutil.rmtree(zarr_path, ignore_errors=False, onerror=None)
    elif Path(zarr_path).exists() and not overwrite:
        logger.warning(
            f"File already exists at {zarr_path}. Set overwrite=True to remove."
        )
        return None

    # write the image data
    store = parse_url(zarr_path, mode="w").store
    root = zarr.group(store=store, overwrite=overwrite)

    # TODO: Add Channel information etc. to the root along those lines
    """
    # add omero metadata_tools: the napari ome-zarr plugin uses this to pass rendering
    # options to napari.
    root.attrs['omero'] = {
        'channels': [{
                'color': 'ffffff',
                'label': 'LS-data',
                'active': True,
                }]
        }

    """

    # write the OME-ZARR file
    ome_zarr.writer.write_image(
        image=array5d,
        group=root,
        axes=axes,
        storage_options=dict(chunks=array5d.shape),
    )

    logger.info(f"Finished writing OME-ZARR to: {zarr_path}")

    return zarr_path

In [5]:
# try to find the folder with data and download otherwise from GitHub.

# Folder containing the input data
if IN_COLAB:
    INPUT_FOLDER = 'data/'
if not IN_COLAB:
    INPUT_FOLDER = '../../data/'

# Path to the data on GitHub
GITHUB_IMAGES_PATH = "https://raw.githubusercontent.com/sebi06/czitools/main/data.zip"

# Download data
if not (os.path.isdir(INPUT_FOLDER)):
    compressed_data = './data.zip'
    if not os.path.isfile(compressed_data):
        import io
        response = requests.get(GITHUB_IMAGES_PATH, stream=True)
        compressed_data = io.BytesIO(response.content)

    import zipfile
    with zipfile.ZipFile(compressed_data, 'r') as zip_accessor:
        zip_accessor.extractall('./')

In [6]:
if IN_COLAB:
    filepath = os.path.join(os.getcwd(), "data/CellDivision_T3_Z5_CH2_X240_Y170.czi")
    zarr_path = Path(filepath[:-4] + ".ome.zarr")

if not IN_COLAB:
    defaultdir = os.path.join(Path(os.getcwd()).resolve().parents[1], "data")
    filepath = os.path.join(defaultdir, "CellDivision_T3_Z5_CH2_X240_Y170.czi")
    zarr_path = defaultdir / Path(filepath[:-4] + ".ome.zarr")

logger.info(zarr_path)

# check if path exists
remove = True
if zarr_path.exists() and remove:
    shutil.rmtree(zarr_path, ignore_errors=False, onerror=None)

2025-08-25 15:18:12,909 - czitools - INFO - /datadisk1/Github/czitools/data/CellDivision_T3_Z5_CH2_X240_Y170.ome.zarr


In [9]:
# get the metadata at once as one big class
mdata = czimd.CziMetadata(filepath)
logger.info(f"Number of Scenes: {mdata.image.SizeS}")
scene_id = 0

array, mdata = read_tools.read_6darray(filepath)

array = array[scene_id, ...]
logger.info(f"Array Shape: {array.shape}")

Reading sublocks planes: 0 2Dplanes [00:00, ? 2Dplanes/s]

2025-08-25 15:19:51,505 - czitools - INFO - Number of Scenes: None


Reading sublocks planes: 0 2Dplanes [00:00, ? 2Dplanes/s]

Reading 2D planes: 0 2Dplanes [00:00, ? 2Dplanes/s]

2025-08-25 15:19:51,812 - czitools - INFO - Array Shape: (3, 2, 5, 170, 240)


In [10]:
# Approach 1: Use ome-zarr-py to write OME-ZARR
zarr_path1 = Path(str(filepath)[:-4] + "_1.ome.zarr")

# write OME-ZARR using utility function
zarr_path1 = write_omezarr(
    array, zarr_path=str(zarr_path1), axes="tczyx", overwrite=True
)

logger.info(f"Written OME-ZARR using ome-zarr.py: {zarr_path1}")

2025-08-25 15:20:13,717 - czitools - INFO - Finished writing OME-ZARR to: /datadisk1/Github/czitools/data/CellDivision_T3_Z5_CH2_X240_Y170_1.ome.zarr
2025-08-25 15:20:13,717 - czitools - INFO - Written OME-ZARR using ome-zarr.py: /datadisk1/Github/czitools/data/CellDivision_T3_Z5_CH2_X240_Y170_1.ome.zarr


In [11]:
# Approach 2: Use ngff-zarr to create NGFF structure and write using ome-zarr-py
zarr_path2 = Path(str(filepath)[:-4] + "_2.ome.zarr")

# create NGFF image from the array
image = nz.to_ngff_image(array.data,
                         dims=["t", "c", "z", "y", "x"],
                         scale={"y": mdata.scale.Y, "x": mdata.scale.X, "z": mdata.scale.Z},
                         name=mdata.filename)

# create multi-scaled, chunked data structure from the image
multiscales = nz.to_multiscales(image, [2, 4], method=nz.Methods.DASK_IMAGE_GAUSSIAN)

# write using ngff-zarr
nz.to_ngff_zarr(zarr_path2, multiscales)
logger.info(f"NGFF Image: {image}")
logger.info(f"Written OME-ZARR using ngff-zarr: {zarr_path2}")

2025-08-25 15:20:19,409 - czitools - INFO - NGFF Image: NgffImage(data=dask.array<rechunk-merge, shape=(3, 2, 5, 170, 240), dtype=uint16, chunksize=(1, 2, 5, 128, 128), chunktype=numpy.ndarray>, dims=['t', 'c', 'z', 'y', 'x'], scale={'y': np.float64(0.091), 'x': np.float64(0.091), 'z': np.float64(0.32)}, translation={'z': 0.0, 'y': 0.0, 'x': 0.0}, name='CellDivision_T3_Z5_CH2_X240_Y170.czi', axes_units=None, axes_orientations=None, computed_callbacks=[])
2025-08-25 15:20:19,409 - czitools - INFO - Written OME-ZARR using ngff-zarr: /datadisk1/Github/czitools/data/CellDivision_T3_Z5_CH2_X240_Y170_2.ome.zarr
